Datu iegūšana




In [2]:
import pandas as pd
import time

# URL vārdnīca ar sezonām un to saiti uz attiecīgo FBref lapu
urls = {
    "2020-2021": "https://fbref.com/en/comps/9/2020-2021/schedule/2020-2021-Premier-League-Scores-and-Fixtures",
    "2021-2022": "https://fbref.com/en/comps/9/2021-2022/schedule/2021-2022-Premier-League-Scores-and-Fixtures",
    "2022-2023": "https://fbref.com/en/comps/9/2022-2023/schedule/2022-2023-Premier-League-Scores-and-Fixtures",
    "2023-2024": "https://fbref.com/en/comps/9/2023-2024/schedule/2023-2024-Premier-League-Scores-and-Fixtures",
    "2024-2025": "https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures"
}

# Saraksts, kur glabāsim DataFrame ar katras sezonas datiem
all_seasons_data = []

# Datums pēc kura spēles netiks ņemtas vērā
cutoff_date = pd.to_datetime("2025-05-24")

# Apstrādājam katru sezonu pēc kārtas
for season, url in urls.items():
    print(f"Apstrādā sezonas {season} datus...")
    try:
        # Nolasām visas tabulas no norādītā URL
        tables = pd.read_html(url)

        # Parasti pirmajā tabulā ir nepieciešamie dati
        df = tables[0]

        # Pievienojam kolonu ar sezonas identifikatoru
        df['Season'] = season

        # Izmetam rindas, kur 'Wk' ir NaN (tās ir atdalītājrindas)
        df = df[df['Wk'].notna()].copy()

        # Ja ir 'Date' kolonna, pārveidojam to uz datetime un filtrējam pēc datuma
        if 'Date' in df.columns:
            df.loc[:, 'Date'] = pd.to_datetime(df['Date'], errors='coerce')
            df = df[df['Date'] < cutoff_date]  # Atstāj tikai spēles pirms cutoff datuma

        # Ja xG kolonnas ir ar dažādiem nosaukumiem, pārdēvējam tās
        if 'xG' in df.columns and 'xG.1' in df.columns:
            df = df.rename(columns={'xG': 'xG_Home', 'xG.1': 'xG_Away'})

        # Izmetam nevajadzīgās kolonnas, ja tās eksistē
        columns_to_drop = ['Match Report', 'Notes']
        df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

        # Score kolonnu, formatējam un sadalām pa mājinieku un viesu vārtiem
        if 'Score' in df.columns:
            df['Score'] = df['Score'].fillna('0-0').astype(str)
            df['Score'] = df['Score'].str.replace('–', '-', regex=False)
            df[['Home Goals', 'Away Goals']] = df['Score'].str.split('-', expand=True)
            # Pārveidojam vārtu kolonnas uz skaitļiem
            df['Home Goals'] = pd.to_numeric(df['Home Goals'], errors='coerce')
            df['Away Goals'] = pd.to_numeric(df['Away Goals'], errors='coerce')

        # Pievienojam apstrādāto sezonas DataFrame galvenajam sarakstam
        all_seasons_data.append(df)

        # Pieliekam pauzi, lai nesanāktu iztērēt visas servera pieprasījuma iespējas
        time.sleep(2)

    except Exception as e:
        print(f"Kļūda apstrādājot {season} datus: {e}")

# Ja vismaz kādi dati ir apstrādāti, apvienojam tos vienotā sarakstā
if all_seasons_data:
    combined_df = pd.concat(all_seasons_data, ignore_index=True)

    # Pārveidojam rindu indeksus uz rindu kārtas numuriem
    combined_df = combined_df.reset_index(drop=True)

    # Saglabājam visu sezonu apvienoto CSV failā
    combined_df.to_csv("Premier_League_All_Seasons_xG.csv", index=False)

    # Saglabājam katras sezonas failu atsevišķi
    for season, df in zip(urls.keys(), all_seasons_data):
        df.to_csv(f"Premier_League_{season}_xG.csv", index=False)

    # Izdrukājam pirmās piecas rindiņas un rindu skaitu
    print("\nApvienoto datu priekšskats:")
    print(combined_df.head())
    print(f"\nKopā rindu: {len(combined_df)}")
else:
    print("Neviena sezona netika veiksmīgi apstrādāta.")

Apstrādā sezonu 2020-2021 datus...
Apstrādā sezonu 2021-2022 datus...
Apstrādā sezonu 2022-2023 datus...
Apstrādā sezonu 2023-2024 datus...
Apstrādā sezonu 2024-2025 datus...

Apvienoto datu priekšskats:
    Wk  Day                 Date   Time            Home  xG_Home Score  \
0  1.0  Sat  2020-09-12 00:00:00  12:30          Fulham      0.1   0-3   
1  1.0  Sat  2020-09-12 00:00:00  15:00  Crystal Palace      1.1   1-0   
2  1.0  Sat  2020-09-12 00:00:00  17:30       Liverpool      2.7   4-3   
3  1.0  Sat  2020-09-12 00:00:00  20:00        West Ham      1.0   0-2   
4  1.0  Sun  2020-09-13 00:00:00  14:00       West Brom      0.4   0-3   

   xG_Away            Away  Attendance           Venue         Referee  \
0      1.9         Arsenal         NaN  Craven Cottage  Chris Kavanagh   
1      0.9     Southampton         NaN   Selhurst Park   Jonathan Moss   
2      0.3    Leeds United         NaN         Anfield  Michael Oliver   
3      1.6   Newcastle Utd         NaN  London Stadium 

Dati tiek saglabāti google drive


In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
# Save to a folder in Google Drive
combined_df.to_csv("/content/drive/MyDrive/bakalauraDarbs/Premier_League_All_Seasons_xG.csv", index=False)
